In [3]:
import torch
import torch.nn as nn
import time
from tqdm import tqdm
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision.models import resnet18, resnet34, resnet50, resnet101, ResNet18_Weights
import sys
sys.path.append(sys.path[0]+'/../')
from dataloaders.minc_dataloader import MINCDataset
from utils.loss_resnet import LossRN
from models.resnet50_101 import ResNet50, ResNet101
from models.resnet18_34 import ResNet18, ResNet34
from models.fusion_resnet import fusion_resnet

In [4]:
LOAD = True

if LOAD:
    resnet = resnet50().cuda()
    resnet.fc = nn.Linear(resnet.fc.in_features, 12).cuda()
    resnet.load_state_dict(torch.load("../weights/resnet50_12class.pth"), strict=False)
else:
    resnet = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1).cuda()
    resnet.fc = nn.Linear(resnet.fc.in_features, 23).cuda()
e = resnet.eval()

In [5]:
'''if LOAD:
    resnet = fusion_resnet(1).cuda()
    resnet.load_state_dict(torch.load("../weights/fusion_resnet_df.pth"), strict=False)
else:
    resnet = fusion_resnet(1).cuda()
e = resnet.eval()'''

'if LOAD:\n    resnet = fusion_resnet(1).cuda()\n    resnet.load_state_dict(torch.load("../weights/fusion_resnet_df.pth"), strict=False)\nelse:\n    resnet = fusion_resnet(1).cuda()\ne = resnet.eval()'

In [6]:
min_path = '/home/ahmed/workspace/notebook/matrec/datasets/minc'
labels_path = '/home/ahmed/workspace/notebook/matrec/datasets/minc/train.txt'
labels_path_t = '/home/ahmed/workspace/notebook/matrec/datasets/minc/test.txt'
BATCH_SIZE = 8
TRAIN_ITER = 1000
TEST_ITER = 400

dataset_train = MINCDataset(min_path, labels_path)
train_loader = DataLoader(dataset=dataset_train, batch_size=BATCH_SIZE, num_workers=0, pin_memory=False, shuffle=True)

dataset_test = MINCDataset(min_path, labels_path_t)
test_loader = DataLoader(dataset=dataset_test, batch_size=BATCH_SIZE, num_workers=0, pin_memory=False, shuffle=True)

optimizer = torch.optim.Adam(resnet.parameters(), lr=0.0001)
loss = LossRN(True)

In [7]:
for epc in range(1,3):
    r = tqdm(enumerate(train_loader), leave=False, desc="Epoch: {}".format(epc), total=len(train_loader))
    for idx, (x, y) in r:
        y_pred = resnet(x.cuda())
        lf = loss.compute(y_pred, y)
        optimizer.zero_grad()
        lf.backward()
        optimizer.step()
        r.set_postfix(loss=lf.item())
        if (idx+1) % TRAIN_ITER == 0:
            #save checkpoint
            torch.save(resnet.state_dict(), "../weights/resnet50_12class.pth")
            #test loss run on val
            with torch.no_grad():
                #test_save = open("../logs/test_acc_fmdf.txt", "a+t")
                ac = 0
                for i in range(TEST_ITER):
                    x, y = next(iter(test_loader))
                    y_pred = resnet(x.cuda())
                    ac = ac + loss.accuracy(y_pred, y)
                ac = ac/TEST_ITER
                print('Test N ' + str(epc) + str(idx//TRAIN_ITER) + ' : ' + str(float(ac)) + '\n')
                #test_save.write('Test N ' + str(epc) + str(idx//TRAIN_ITER) + ' : ' + str(float(ac)) + '\n')
                #test_save.close()

KeyboardInterrupt: 